##### Copyright 2023 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PaLM API: Text Quickstart with Python

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://developers.generativeai.google/tutorials/text_quickstart"><img src="https://developers.generativeai.google/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on Generative AI</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tutorials/text_quickstart.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this notebook, you'll learn how to get started with the PaLM API, which gives you access to Google's latest large language models. Here, you'll learn how to use the PaLM API's text generation features.

## Setup

**Note**: At this time, the PaLM API is only available in the US.

First, download and install the PaLM API Python library.

In [1]:
!pip install -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.3/113.3 kB 14.0 MB/s eta 0:00:00


In [2]:
import pprint
import google.generativeai as palm

### Grab an API Key

To get started, you'll need to [create an API key](https://developers.generativeai.google/tutorials/setup).

In [3]:
palm.configure(api_key='ENTER YOUR API KEY HERE')

## Text generation

Use the `palm.list_models` function to find available models:

In [4]:
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model = models[0].name
print(model)

models/text-bison-001


Use the `palm.generate_text` method to generate text:

In [5]:
prompt = """
You are an expert at solving word problems.

Solve the following problem:

I have three houses, each with three cats.
each cat owns 4 mittens, and a hat. Each mitten was
knit from 7m of yarn, each hat from 4m.
How much yarn was needed to make all the items?

Think about it step by step, and show your work.
"""

completion = palm.generate_text(
    model=model,
    prompt=prompt,
    temperature=0,
    # The maximum length of the response
    max_output_tokens=800,
)

print(completion.result)

There are 3 houses * 3 cats / house = 9 cats. So, 9 cats * 4 mittens / cat = 36 mittens were made. Also, 9 cats * 1 hat / cat = 9 hats were made. So, 36 mittens * 7m / mitten = 252m of yarn was used for the mittens. Also, 9 hats * 4m / hat = 36m of yarn was used for the hats. In total, 252m + 36m = 288m of yarn was used.
Thus, the answer is 288.


## More options

The `palm.generate_text` function has a few other arguments worth mentioning.

### Stop sequences

Use the `stop_sequences` argument to stop generation early. 

For example LLM's often make mistakes in arithmetic. You could ask the model to "use a calculator" by putting equations in a `<calc>` tag. 

Have the model stop at the closing tag, so you can edit the prompt:

In [6]:
calc_prompt = f"""
Please solve the following problem.

{prompt}

----------------

Important: Use the calculator for each step.
Don't do the arithmetic in your head. 

To use the calculator wrap an equation in <calc> tags like this: 

<calc> 3 cats * 2 hats/cat </calc> = 6

----------------

"""

equation=None
while equation is None:
    completion = palm.generate_text(
        model=model,
        prompt=calc_prompt,
        stop_sequences=['</calc>'],
        # The maximum length of the response
        max_output_tokens=800,
    )

    try:
        response, equation = completion.result.split('<calc>', maxsplit=1)
    except Exception:
        continue

In [7]:
print(response)

Chain-of-thought:

Three houses have 3 * 3 = 9 cats. They have 9 * 4 = 36 mittens. They have 9 * 2 = 18 hats. This means they needed 


In [8]:
print(equation)

 36 * 7 


From there you can calculate the result, and assemble a new prompt for the model to continue from.
For a complete working implementation see the [Text calculator example](../examples/text_calculator.ipynb).

### Candidates

Typically, there's some degree of randomness in the text produced by LLMs. (Read more about why in the [LLM primer](https://developers.generativeai.google/guide/concepts)). That means that when you call the API more than once with the same input, you might get different responses. You can use this feature to your advantage to get alternate model responses.

The `temperature` argument controls the variance of the responses. The `palm.Model` object gives the default value for `temperature` and other parameters.

In [9]:
models[0]

Model(name='models/text-bison-001', base_model_id='', version='001', display_name='Text Bison', description='Model targeted for text generation.', input_token_limit=8196, output_token_limit=1024, supported_generation_methods=['generateText'], temperature=0.7, top_p=0.95, top_k=40)

The `candidate_count` argument controls the number of responses returned:

In [10]:
completion = palm.generate_text(
    model=model,
    prompt=prompt,
    # The number of candidates to return
    candidate_count=8,
    # Set the temperature to 1.0 for more variety of responses.
    temperature=1.0,
    max_output_tokens=800,
)

print(completion.result)

In each house, there are 3 cats * 4 mittens / cat = 12 mittens. So, for all three houses, there are 3 houses * 12 mittens / house = 36 mittens. And for all three houses, there are 3 houses * 3 cats / house * 1 hat / cat = 9 hats. So, the total amount of yarn needed is 36 mittens * 7m / mitten + 9 hats * 4m / hat = 252m + 36m = 288m.
Thus, the answer is 288.


When you request multiple candidates the `Completion.result` attribute still just contains the first one. The `Completion.candidates` attribute contains all of them:

In [11]:
import pprint
pprint.pprint(completion.candidates)

[{'output': 'In each house, there are 3 cats * 4 mittens / cat = 12 mittens. '
            'So, for all three houses, there are 3 houses * 12 mittens / house '
            '= 36 mittens. And for all three houses, there are 3 houses * 3 '
            'cats / house * 1 hat / cat = 9 hats. So, the total amount of yarn '
            'needed is 36 mittens * 7m / mitten + 9 hats * 4m / hat = 252m + '
            '36m = 288m.\n'
            'Thus, the answer is 288.',
  'safety_ratings': [{'category': <HarmCategory.HARM_CATEGORY_DEROGATORY: 1>,
                      'probability': <HarmProbability.NEGLIGIBLE: 1>},
                     {'category': <HarmCategory.HARM_CATEGORY_TOXICITY: 2>,
                      'probability': <HarmProbability.NEGLIGIBLE: 1>},
                     {'category': <HarmCategory.HARM_CATEGORY_VIOLENCE: 3>,
                      'probability': <HarmProbability.NEGLIGIBLE: 1>},
                     {'category': <HarmCategory.HARM_CATEGORY_SEXUAL: 4>,
                 

So, since you know the answer to this problem, it's easy to check the solve rate:

In [12]:
import numpy as np
np.mean(['288' in c['output'] for c in completion.candidates])

0.75